# 4회차 실습 과제 — 정답

## 문제 항목

| 문제     | 주제                         |
|----------|------------------------------|
| 문제 1   | 편의점 일일 매출 분석        |
| 문제 2   | 두 생산라인 제품 무게 비교   |
| 문제 3   | 요일별 고객 방문 패턴 분석   |
| 문제 4   | 연령대별 운동 선호도 조사    |

---

## 핵심 공식 정리

### 가설검정 프로세스

| 단계 | 내용                                   | 비고                                          |
|------|----------------------------------------|-----------------------------------------------|
| 1    | **분석 목표** 파악                     | 무엇을 검정하려 하는지 명확히 정의합니다       |
| 2    | **가설 설정** (H₀, H₁)                | 귀무가설과 대립가설을 수식으로 표현합니다       |
| 3    | **검정 방향** 결정                     | 양측/단측 검정을 선택하고 근거를 제시합니다     |
| 4    | **가정 검정**                          | 정규성, 등분산, 기대빈도 조건 등을 확인합니다   |
| 5    | **검정 수행**                          | 가정 결과에 따라 적절한 검정을 선택합니다       |
| 6    | **효과크기** 산출                      | 통계적 유의성과 별개로 실질적 의미를 판단합니다 |
| 7    | **결론** 도출                          | p-value와 효과크기를 종합하여 해석합니다        |

> 카이제곱 검정(적합도, 독립성)은 검정 자체가 "차이 존재 여부"를 판정하므로
> 양측/단측 개념이 적용되지 않습니다 (3단계 생략).

### 양측검정 vs 단측검정

| 구분          | 양측검정 (two-sided)           | 단측검정 (one-sided)              |
|---------------|--------------------------------|-----------------------------------|
| H₁ 형태      | μ ≠ μ₀                        | μ > μ₀ 또는 μ < μ₀              |
| 사용 시기     | 방향을 사전에 특정할 수 없을 때 | 이론적·실무적 근거로 방향이 명확할 때 |
| 검정력        | 상대적으로 낮음                | 같은 α에서 더 높은 검정력         |
| 주의          | 가장 보수적이고 안전한 선택     | 사전 근거 없이 사용하면 부적절     |

### 정규성 검정

| 방법            | 판정 기준                          | 비고                              |
|-----------------|------------------------------------|-----------------------------------|
| Shapiro-Wilk    | p > 0.05 → 정규성 기각 못 함      | 소표본에 적합, 대표본에서 과민    |
| Q-Q Plot        | 점들이 대각선 위 → 정규            | 시각적 판단, Shapiro-Wilk와 함께  |

### 검정 선택 가이드 (연속형)

| 상황            | 정규성 충족                     | 정규성 위반                   |
|-----------------|---------------------------------|-------------------------------|
| 단일·대응표본   | 단일/대응표본 t-검정            | Wilcoxon signed-rank          |
| 독립 2집단      | Student's t (등분산) / Welch's t (이분산) | Mann-Whitney U        |

### 비모수 검정 효과크기

| 효과크기         | 적용 검정               | 해석 기준 (small / medium / large) |
|------------------|-------------------------|------------------------------------|
| rank-biserial r  | Wilcoxon, Mann-Whitney  | 0.1 / 0.3 / 0.5                   |

### 카이제곱 검정

| 검정           | 공식                                                          | 자유도                   |
|----------------|---------------------------------------------------------------|--------------------------|
| 적합도 검정    | $\chi^2 = \sum \frac{(O_i - E_i)^2}{E_i}$                   | $df = k - 1$            |
| 독립성 검정    | $\chi^2 = \sum \frac{(O_{ij} - E_{ij})^2}{E_{ij}}$          | $df = (r-1)(c-1)$       |
| 기대빈도       | $E_{ij} = \frac{R_i \times C_j}{N}$                          | —                        |

### 효과크기 (카이제곱)

| 효과크기     | 적용 검정      | 공식                                        | 해석 기준 (small / medium / large) |
|--------------|----------------|---------------------------------------------|------------------------------------|
| Cohen's w    | 적합도         | $\sqrt{\chi^2 / N}$                        | 0.1 / 0.3 / 0.5                   |
| Cramér's V   | 독립성         | $\sqrt{\chi^2 / (N \cdot \min(r-1, c-1))}$ | df*에 따라 다름                    |
| φ (Phi)      | 독립성 (2×2)   | $\sqrt{\chi^2 / N}$                        | 0.1 / 0.3 / 0.5                   |

### Cochran's rule (기대빈도 조건)

| 조건 | 기준                                     | 위반 시 대안                              |
|------|------------------------------------------|-------------------------------------------|
| (1)  | 기대빈도 < 5인 셀이 전체의 20% 이하     | 적합도 → Monte Carlo / 범주 병합          |
| (2)  | 기대빈도 < 1인 셀이 0개                  | 독립성(2×2) → Fisher / 독립성(R×C) → FFH  |

### 판정 규칙

| p-value 결과  | 판정             | 해석                                          |
|---------------|------------------|-----------------------------------------------|
| p ≤ α         | H₀ **기각**      | "이 결과가 우연이라고 보기 어렵습니다" → 유의   |
| p > α         | H₀ **기각 실패** | "증거 부족, 판단 보류" (H₀가 참은 아닙니다!)    |

---

In [ ]:
# 필수 라이브러리 Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.stats.contingency_tables import Table
import pingouin as pg
import warnings
import platform

warnings.filterwarnings('ignore')

try:
    from IPython.display import display
except ImportError:
    display = lambda x: print(x.to_string() if hasattr(x, 'to_string') else x)

# 운영체제별 한글 폰트 설정
if platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'
elif platform.system() == 'Darwin':
    plt.rcParams['font.family'] = 'AppleGothic'
else:
    plt.rcParams['font.family'] = 'NanumGothic'

plt.rcParams['axes.unicode_minus'] = False

print("=" * 60)
print("4회차 실습 과제 — 정답")
print("=" * 60)


---

## 문제 1: 편의점 일일 매출 분석

한 편의점 본사는 특정 지점의 일일 매출이 전국 평균 **250만원**과 다른지 확인하려 합니다.

20일간의 매출 데이터를 수집했는데, 매출 분포가 정규분포를 따르는지 먼저 확인해야 합니다.

**분석 목표**: 이 편의점의 일일 매출이 전국 평균(250만원)과 통계적으로 유의한 차이가 있는지 검정합니다.

**주어진 데이터:**

In [ ]:
print("\n[문제 1] 편의점 일일 매출 분석")
print("=" * 50)

np.random.seed(501)
daily_sales = np.round(np.random.lognormal(mean=5.5, sigma=0.3, size=20), 1)
mu0_sales = 250.0  # 전국 평균 매출 (만원)

print(f"일일 매출 데이터 (n = {len(daily_sales)}일, 단위: 만원):")
print(daily_sales)
print(f"\n표본 평균: {np.mean(daily_sales):.1f}만원")
print(f"표본 중앙값: {np.median(daily_sales):.1f}만원")
print(f"전국 평균(μ₀): {mu0_sales}만원")


### 문제 1-1: 정규성 검정 (가정 검정)

**(a)** 수치적 정규성 검정(Shapiro-Wilk)을 수행하세요.

**(b)** Q-Q Plot을 그려 시각적으로 분포를 확인하세요.

**(c)** 두 결과를 종합하여 "정규분포를 따르는가?"를 판단하고,

Q-Q Plot에서 관찰되는 패턴이 어떤 분포 특성을 나타내는지 설명하세요.

In [ ]:
print("\n[문제 1-1] 정규성 검정")
print("-" * 40)

# (a) Shapiro-Wilk 정규성 검정
stat_sw, p_sw = stats.shapiro(daily_sales)

print(f"(a) 정규성 검정:")
print(f"    W = {stat_sw:.4f}")
print(f"    p-value = {p_sw:.4f}")
verdict_sw = "정규성 기각 (p ≤ 0.05)" if p_sw <= 0.05 else "정규성 기각 못 함 (p > 0.05)"
print(f"    판정: {verdict_sw}")


In [ ]:
# (b) Q-Q Plot + 히스토그램
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

stats.probplot(daily_sales, dist="norm", plot=axes[0])
axes[0].set_title("Q-Q Plot", fontsize=12, fontweight="bold")
axes[0].grid(alpha=0.3)

axes[1].hist(daily_sales, bins=8, color="#6366F1", alpha=0.7, edgecolor="white")
axes[1].axvline(np.mean(daily_sales), color="red", linestyle="--", label=f"평균={np.mean(daily_sales):.1f}")
axes[1].axvline(np.median(daily_sales), color="orange", linestyle="-.", label=f"중앙값={np.median(daily_sales):.1f}")
axes[1].set_title("매출 분포", fontsize=12, fontweight="bold")
axes[1].set_xlabel("매출 (만원)")
axes[1].set_ylabel("빈도")
axes[1].legend(fontsize=9)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# (c) 종합 판단
print(f"\n(c) 종합 판단:")
print(f"    Shapiro-Wilk: W={stat_sw:.4f}, p={p_sw:.4f} → {verdict_sw}")
if p_sw <= 0.05:
    print(f"    Q-Q Plot 패턴: 우상단에서 대각선 위로 벗어남 → 오른쪽 꼬리가 긴(right-skewed) 분포")
    print(f"    결론: 정규분포를 따르지 않습니다 → 비모수 검정을 사용해야 합니다")
else:
    print(f"    Q-Q Plot 패턴: 점들이 대각선 근처에 위치하나 양 끝에서 약간 벗어남")
    print(f"    결론: 평균 > 중앙값 → 오른쪽으로 치우친 경향이 있으므로 비모수 검정이 더 적절합니다")


### 문제 1-2: 가설 설정과 검정 수행

**(a)** 분석 목표에 맞는 **귀무가설(H₀)**과 **대립가설(H₁)**을 설정하세요.

**(b)** **양측검정** 또는 **단측검정** 중 어떤 것이 적절한지 결정하고, 그 근거를 설명하세요.

**(c)** 정규성 검정 결과를 바탕으로, 이 데이터에 적절한 검정 방법을 선택하고 그 이유를 설명하세요.

**(d)** 선택한 검정을 수행하세요 (α = 0.05).

**(e)** 효과크기를 계산하세요.

> 모수 검정인지 비모수 검정인지에 따라 적절한 효과크기 지표가 다릅니다.
> 해석 기준과 함께 보고하세요.

**(f)** p-value와 효과크기를 종합하여 최종 결론을 내리세요.

In [ ]:
print("\n[문제 1-2] 가설 설정과 검정 수행")
print("-" * 40)

# (a) 가설 설정
print("(a) 가설 설정:")
print(f"    H₀: 이 편의점의 일일 매출은 전국 평균(μ₀ = 250만원)과 같습니다")
print(f"    H₁: 이 편의점의 일일 매출은 전국 평균(μ₀ = 250만원)과 다릅니다")

# (b) 양측/단측 결정
print(f"\n(b) 검정 방향:")
print(f"    선택: 양측검정")
print(f"    근거: '전국 평균과 다른지'를 확인하는 것이므로 매출이 높은지 낮은지")
print(f"    방향이 사전에 정해지지 않았습니다. 따라서 양측검정이 적절합니다.")

# (c) 검정 방법 선택
print(f"\n(c) 검정 방법 선택:")
if p_sw <= 0.05:
    print(f"    정규성 결과: p={p_sw:.4f} ≤ 0.05 → 정규분포를 따르지 않습니다")
else:
    print(f"    정규성 결과: p={p_sw:.4f} > 0.05이지만, 분포가 비대칭합니다")
print(f"    선택: Wilcoxon signed-rank 검정 (단일표본 비모수)")
print(f"    이유: lognormal 분포 특성(오른쪽 치우침)으로 비모수 검정이 적절합니다")

# (d) Wilcoxon signed-rank 검정
test_stat, p_value = stats.wilcoxon(daily_sales - mu0_sales)

print(f"\n(d) 검정 결과:")
print(f"    검정통계량 W: {test_stat:.1f}")
print(f"    p-value: {p_value:.4f}")
verdict_1 = "H₀ 기각 → 유의한 차이가 있습니다" if p_value <= 0.05 else "H₀ 기각 실패 → 유의한 차이가 없습니다"
print(f"    판정 (α=0.05): {verdict_1}")

# (e) 효과크기: rank-biserial r
result_w = pg.wilcoxon(daily_sales, np.full(len(daily_sales), mu0_sales))
effect_size = result_w['RBC'].values[0]
size_label = '작은' if abs(effect_size) < 0.3 else '중간' if abs(effect_size) < 0.5 else '큰'

print(f"\n(e) 효과크기:")
print(f"    rank-biserial r = {effect_size:.4f}")
print(f"    해석 기준: |r| < 0.1 무시 / 0.1~0.3 작은 / 0.3~0.5 중간 / > 0.5 큰")
print(f"    해석: {size_label} 효과 크기입니다")

# (f) 종합 결론
print(f"\n(f) 종합 결론:")
if p_value <= 0.05:
    print(f"    이 편의점의 일일 매출은 전국 평균 {mu0_sales}만원과 유의하게 다릅니다")
    print(f"    (Wilcoxon signed-rank, W={test_stat:.1f}, p={p_value:.4f}, r={effect_size:.3f})")
else:
    print(f"    이 편의점의 일일 매출은 전국 평균 {mu0_sales}만원과 유의한 차이가 없습니다")
    print(f"    (Wilcoxon signed-rank, W={test_stat:.1f}, p={p_value:.4f}, r={effect_size:.3f})")
    print(f"    표본 평균({np.mean(daily_sales):.1f})은 모평균보다 높지만 통계적으로 유의하지 않습니다")


### 문제 1-3: 모수 vs 비모수 비교

같은 데이터에 대해 **모수 검정과 비모수 검정을 모두** 수행하고, 결과를 비교해 보세요.

**(a)** 모수 검정(단일표본 t-검정)의 결과를 구하세요.

**(b)** 비모수 검정(Wilcoxon signed-rank)의 결과를 구하세요.

**(c)** 두 검정의 결론이 같은가요, 다른가요?

만약 다르다면, 이 데이터의 어떤 특성 때문에 차이가 발생하는지 설명하세요.

In [ ]:
print("\n[문제 1-3] 모수 vs 비모수 비교")
print("-" * 40)

# (a) 모수 검정
t_stat_1, p_t = stats.ttest_1samp(daily_sales, mu0_sales)
print(f"(a) 단일표본 t-검정: t = {t_stat_1:.4f}, p = {p_t:.4f}")
verdict_t = "H₀ 기각" if p_t <= 0.05 else "H₀ 기각 실패"
print(f"    판정: {verdict_t}")

# (b) 비모수 검정
w_stat_1, p_w = stats.wilcoxon(daily_sales - mu0_sales)
print(f"\n(b) Wilcoxon signed-rank: W = {w_stat_1:.1f}, p = {p_w:.4f}")
verdict_w = "H₀ 기각" if p_w <= 0.05 else "H₀ 기각 실패"
print(f"    판정: {verdict_w}")

# (c) 비교 해석
print(f"\n(c) 결과 비교 및 해석:")
print(f"    모수 검정(t): p = {p_t:.4f} → {verdict_t}")
print(f"    비모수 검정(W): p = {p_w:.4f} → {verdict_w}")
if (p_t <= 0.05) != (p_w <= 0.05):
    print(f"    → 두 검정의 결론이 다릅니다!")
    print(f"    차이 원인:")
    print(f"      - 평균({np.mean(daily_sales):.1f}) > 중앙값({np.median(daily_sales):.1f}) → 오른쪽 치우침")
    print(f"      - t-검정은 평균을 기반으로 하므로 이상치/비대칭에 민감합니다")
    print(f"      - Wilcoxon은 순위 기반이므로 이상치에 강건(robust)합니다")
    print(f"      - 비대칭 분포에서는 비모수 검정이 더 신뢰할 수 있습니다")
else:
    print(f"    → 두 검정의 결론이 동일합니다")
    print(f"    그러나 p-value 차이가 있습니다:")
    print(f"      - 평균({np.mean(daily_sales):.1f}) > 중앙값({np.median(daily_sales):.1f}) → 오른쪽 치우침")
    print(f"      - t-검정은 평균에 민감하므로 극단값의 영향을 더 받습니다")
    print(f"      - Wilcoxon은 순위 기반이므로 이상치에 강건합니다")
    print(f"      - 정규성 위반 시에는 비모수 검정의 결과를 우선 신뢰해야 합니다")



---

## 문제 2: 두 생산라인 제품 무게 비교

식품 공장에서 A라인과 B라인에서 생산된 과자의 무게가 동일한지 검증합니다.

A라인은 안정적이지만, B라인은 간헐적 기계 오류로 무게 편차가 큰 것으로 의심됩니다.

**분석 목표**: 두 생산라인(A, B)의 제품 무게에 통계적으로 유의한 차이가 있는지 검정합니다.

**주어진 데이터:**

In [ ]:
print("\n[문제 2] 두 생산라인 제품 무게 비교")
print("=" * 50)

np.random.seed(502)
line_a = np.round(np.random.normal(loc=100, scale=3, size=25), 1)
line_b_normal = np.random.normal(loc=98, scale=2.5, size=20)
line_b_outlier = np.random.normal(loc=110, scale=3, size=5)
line_b = np.round(np.concatenate([line_b_normal, line_b_outlier]), 1)
np.random.shuffle(line_b)

print(f"A라인 (n={len(line_a)}): 평균={np.mean(line_a):.1f}g, 중앙값={np.median(line_a):.1f}g, SD={np.std(line_a, ddof=1):.1f}g")
print(f"B라인 (n={len(line_b)}): 평균={np.mean(line_b):.1f}g, 중앙값={np.median(line_b):.1f}g, SD={np.std(line_b, ddof=1):.1f}g")


In [ ]:
# 시각화: 두 라인 비교
fig, axes = plt.subplots(1, 2, figsize=(13, 5))

axes[0].hist(line_a, bins=10, alpha=0.6, color='#3B82F6', edgecolor='white', label='A라인')
axes[0].hist(line_b, bins=10, alpha=0.6, color='#F59E0B', edgecolor='white', label='B라인')
axes[0].axvline(np.mean(line_a), color='#3B82F6', linestyle='--', linewidth=2)
axes[0].axvline(np.mean(line_b), color='#F59E0B', linestyle='--', linewidth=2)
axes[0].set_xlabel('무게 (g)')
axes[0].set_ylabel('빈도')
axes[0].set_title('두 라인의 제품 무게 분포', fontsize=12, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(alpha=0.3)

bp = axes[1].boxplot([line_a, line_b], labels=['A라인', 'B라인'],
                      patch_artist=True, widths=0.5)
bp['boxes'][0].set_facecolor('#3B82F6')
bp['boxes'][0].set_alpha(0.6)
bp['boxes'][1].set_facecolor('#F59E0B')
bp['boxes'][1].set_alpha(0.6)
axes[1].set_ylabel('무게 (g)')
axes[1].set_title('라인별 무게 비교', fontsize=12, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()


### 문제 2-1: 가설 설정 및 가정 검정

**(a)** 분석 목표에 맞는 **귀무가설(H₀)**과 **대립가설(H₁)**을 설정하세요.

**(b)** **양측검정** 또는 **단측검정** 중 어떤 것이 적절한지 결정하고, 그 근거를 설명하세요.

**(c)** 각 라인의 정규성을 검정하세요.

**(d)** 두 라인의 Q-Q Plot을 나란히 그리세요.

**(e)** 정규성이 충족된다면, 등분산 검정도 수행하세요.

**(f)** 가정 검정 결과를 종합하여 어떤 검정을 사용할지 결정하세요.

의사결정 과정을 단계별로 설명하세요.

In [ ]:
print("\n[문제 2-1] 가설 설정 및 가정 검정")
print("-" * 40)

# (a) 가설 설정
print("(a) 가설 설정:")
print(f"    H₀: 두 생산라인의 제품 무게에 차이가 없습니다 (μ_A = μ_B)")
print(f"    H₁: 두 생산라인의 제품 무게에 차이가 있습니다 (μ_A ≠ μ_B)")

# (b) 양측/단측 결정
print(f"\n(b) 검정 방향:")
print(f"    선택: 양측검정")
print(f"    근거: '무게가 동일한지 검증'하는 것이므로 어느 라인이 더 무거운지")
print(f"    방향이 사전에 정해지지 않았습니다. 따라서 양측검정이 적절합니다.")

# (c) 정규성 검정
stat_a, p_a = stats.shapiro(line_a)
stat_b, p_b = stats.shapiro(line_b)

print(f"\n(c) 정규성 검정:")
print(f"    A라인: W={stat_a:.4f}, p={p_a:.4f} → {'정규' if p_a > 0.05 else '비정규'}")
print(f"    B라인: W={stat_b:.4f}, p={p_b:.4f} → {'정규' if p_b > 0.05 else '비정규'}")


In [ ]:
# (d) Q-Q Plot
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

stats.probplot(line_a, dist="norm", plot=axes[0])
axes[0].set_title("A라인 Q-Q Plot", fontsize=12, fontweight="bold")
axes[0].grid(alpha=0.3)

stats.probplot(line_b, dist="norm", plot=axes[1])
axes[1].set_title("B라인 Q-Q Plot", fontsize=12, fontweight="bold")
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# (e) 등분산 검정
lev_stat, lev_p = stats.levene(line_a, line_b)

print(f"(e) 등분산 검정:")
print(f"    F={lev_stat:.4f}, p={lev_p:.4f}")
verdict_lev = "등분산 가정 유지" if lev_p > 0.05 else "이분산 (등분산 기각)"
print(f"    판정: {verdict_lev}")

# (f) 검정 방법 결정
print(f"\n(f) 검정 방법 결정:")
print(f"    [1] 정규성 — A: p={p_a:.4f} ({'통과' if p_a > 0.05 else '불통과'}), B: p={p_b:.4f} ({'통과' if p_b > 0.05 else '불통과'})")
print(f"    [2] 등분산 — p={lev_p:.4f} ({verdict_lev})")

if p_a > 0.05 and p_b > 0.05:
    if lev_p > 0.05:
        chosen_test = "Student's t-검정 (정규 + 등분산)"
    else:
        chosen_test = "Welch's t-검정 (정규 + 이분산)"
else:
    chosen_test = "Mann-Whitney U 검정 (정규성 위반)"
print(f"    [3] 결론 → 사용할 검정: {chosen_test}")


### 문제 2-2: 검정 수행 및 효과크기

**(a)** 2-1에서 결정한 검정을 수행하세요 (α = 0.05).

**(b)** 적절한 효과크기를 계산하세요.

> 모수 검정(t-검정)이면 Cohen's d를, 비모수 검정이면 rank-biserial r을 계산합니다.

**(c)** p-value와 효과크기를 종합하여 최종 결론을 작성하세요.

In [ ]:
print("\n[문제 2-2] 검정 수행 및 효과크기")
print("-" * 40)

# (a) 검정 수행 (가정 검정 결과에 따라 선택)
if p_a > 0.05 and p_b > 0.05:
    # 정규성 충족 → t-검정
    eq_var = lev_p > 0.05
    test_stat_2, p_value_2 = stats.ttest_ind(line_a, line_b, equal_var=eq_var)
    test_name = "Student's t" if eq_var else "Welch's t"
    print(f"(a) {test_name}-검정 결과:")
    print(f"    t = {test_stat_2:.4f}")
    print(f"    p-value = {p_value_2:.4f}")

    # (b) Cohen's d
    n1, n2 = len(line_a), len(line_b)
    s_pooled = np.sqrt(((n1 - 1) * np.var(line_a, ddof=1) + (n2 - 1) * np.var(line_b, ddof=1)) / (n1 + n2 - 2))
    effect_size_2 = (np.mean(line_a) - np.mean(line_b)) / s_pooled
    size_label_2 = '작은' if abs(effect_size_2) < 0.5 else '중간' if abs(effect_size_2) < 0.8 else '큰'
    print(f"\n(b) Cohen's d = {effect_size_2:.4f} ({size_label_2} 효과)")
    print(f"    해석 기준: |d| < 0.2 무시 / 0.2~0.5 작은 / 0.5~0.8 중간 / > 0.8 큰")
else:
    # 정규성 위반 → Mann-Whitney U
    test_stat_2, p_value_2 = stats.mannwhitneyu(line_a, line_b, alternative='two-sided')
    print(f"(a) Mann-Whitney U 검정 결과:")
    print(f"    U = {test_stat_2:.1f}")
    print(f"    p-value = {p_value_2:.4f}")

    # (b) rank-biserial r
    result_mw = pg.mwu(line_a, line_b)
    effect_size_2 = result_mw['RBC'].values[0]
    size_label_2 = '작은' if abs(effect_size_2) < 0.3 else '중간' if abs(effect_size_2) < 0.5 else '큰'
    print(f"\n(b) rank-biserial r = {effect_size_2:.4f} ({size_label_2} 효과)")
    print(f"    해석 기준: |r| < 0.1 무시 / 0.1~0.3 작은 / 0.3~0.5 중간 / > 0.5 큰")

verdict_2 = "H₀ 기각 → 두 라인 간 유의한 차이" if p_value_2 <= 0.05 else "H₀ 기각 실패 → 유의한 차이 없음"
print(f"    판정 (α=0.05): {verdict_2}")

# (c) 종합 결론
print(f"\n(c) 종합 결론:")
print(f"    p-value = {p_value_2:.4f}, 효과크기 = {effect_size_2:.4f}")
if p_value_2 <= 0.05:
    print(f"    두 생산라인의 제품 무게에 통계적으로 유의한 차이가 있습니다")
else:
    print(f"    두 생산라인의 제품 무게에 통계적으로 유의한 차이가 없습니다")
    print(f"    B라인의 이상치가 평균을 끌어올려 A라인과 유사한 평균을 보이지만,")
    print(f"    중앙값({np.median(line_b):.1f})은 A라인({np.median(line_a):.1f})과 차이가 있을 수 있습니다")


### 문제 2-3: 모수 vs 비모수 — 왜 가정 검정이 중요한가?

**(a)** 같은 데이터에 대해 Welch's t-검정과 Mann-Whitney U 검정을 **모두** 수행하고 결과를 비교하세요.

**(b)** 두 검정의 결론이 다르다면, 어떤 검정의 결과를 더 신뢰해야 하는지

B라인의 분포 특성과 연결하여 설명하세요.

**(c)** "표본이 크면 t-검정을 써도 괜찮다"는 주장에 대해,

이 데이터의 B라인처럼 이상치가 포함된 경우에도 성립하는지 의견을 작성하세요.

In [ ]:
print("\n[문제 2-3] 모수 vs 비모수 비교")
print("-" * 40)

# (a) 두 검정 모두 수행
t_stat_comp, p_t_comp = stats.ttest_ind(line_a, line_b, equal_var=False)
u_stat_comp, p_u_comp = stats.mannwhitneyu(line_a, line_b, alternative='two-sided')

print(f"(a) 검정 결과 비교:")
print(f"    Welch's t: t={t_stat_comp:.4f}, p={p_t_comp:.4f} → {'기각' if p_t_comp <= 0.05 else '기각 실패'}")
print(f"    Mann-Whitney U: U={u_stat_comp:.1f}, p={p_u_comp:.4f} → {'기각' if p_u_comp <= 0.05 else '기각 실패'}")

# (b) 해석
print(f"\n(b) 어떤 검정을 더 신뢰해야 하나요?")
print(f"    B라인은 이상치(~110g)가 포함된 혼합 분포입니다")
print(f"    Welch's t-검정은 평균을 기반으로 하여 이상치에 민감합니다")
print(f"    Mann-Whitney U는 순위 기반이므로 이상치에 강건(robust)합니다")
print(f"    → B라인처럼 이상치가 있는 데이터에서는 Mann-Whitney U가 더 신뢰할 수 있습니다")

# (c) CLT와 이상치
print(f"\n(c) 이상치가 있을 때 t-검정의 한계:")
print(f"    CLT(중심극한정리)는 n이 충분히 크면 표본평균이 정규분포에 근사한다고 합니다")
print(f"    그러나 이상치가 있으면:")
print(f"    (1) 평균이 극단값 쪽으로 왜곡되어 대표성이 떨어집니다")
print(f"    (2) 표준편차가 과대추정되어 검정력이 떨어집니다")
print(f"    (3) B라인(n=25)은 표본이 크지 않아 CLT 효과도 제한적입니다")
print(f"    → 이상치가 포함된 경우에는 표본 크기와 관계없이 비모수 검정이 더 적절합니다")



---

## 문제 3: 요일별 고객 방문 패턴 분석

한 음식점 사장이 "요일에 따라 방문 고객 수가 다르지 않다"고 주장합니다.

한 주간 방문 고객 수를 조사하여 이 주장을 검증합니다.

총 **420명**의 고객이 방문했습니다.

**분석 목표**: 요일별 고객 방문 비율이 균등(각 1/7)한지 검정합니다.

**주어진 데이터:**

In [ ]:
print("\n[문제 3] 요일별 고객 방문 패턴 분석")
print("=" * 50)

np.random.seed(503)
days = ['월', '화', '수', '목', '금', '토', '일']
true_probs = [0.10, 0.10, 0.12, 0.12, 0.16, 0.22, 0.18]
visits = np.random.choice(days, size=420, p=true_probs)
observed_visits = np.array([np.sum(visits == d) for d in days])

print(f"요일별 방문 고객 수 (총 {observed_visits.sum()}명):")
df_visits = pd.DataFrame({'요일': days, '관측 빈도': observed_visits})
display(df_visits)


### 문제 3-1: 가설 설정 및 적합도 검정

**(a)** 분석 목표에 맞는 **귀무가설(H₀)**과 **대립가설(H₁)**을 설정하세요.

> 카이제곱 검정은 "차이가 존재하는가?"를 판정하므로 양측/단측 개념이 적용되지 않습니다.

**(b)** 기대빈도를 계산하고, Cochran's rule을 확인하세요.

기대빈도 < 5인 셀이 전체의 20%를 초과하거나 기대빈도 < 1인 셀이 있는지 점검하세요.

**(c)** 적합도 검정을 수행하세요 (α = 0.05).

**(d)** 결론을 내리세요.

In [ ]:
print("\n[문제 3-1] 가설 설정 및 적합도 검정")
print("-" * 40)

# (a) 가설 설정
print("(a) 가설 설정:")
print(f"    H₀: 요일별 방문 비율이 균등합니다 (p₁ = p₂ = ... = p₇ = 1/7)")
print(f"    H₁: 적어도 하나의 요일이 다른 비율을 보입니다 (모든 pᵢ가 동일하지 않습니다)")
print(f"    * 카이제곱 적합도 검정은 양측/단측 개념이 적용되지 않습니다")

# (b) 기대빈도 + Cochran's rule
n_total_visits = observed_visits.sum()
expected_visits = np.full(len(days), n_total_visits / len(days))

print(f"\n(b) 기대빈도: {expected_visits[0]:.1f} (모든 요일 동일)")
n_below_5 = np.sum(expected_visits < 5)
n_below_1 = np.sum(expected_visits < 1)
pct_below_5 = n_below_5 / len(expected_visits) * 100
print(f"    기대빈도 < 5인 셀: {n_below_5}개 ({pct_below_5:.0f}%)")
print(f"    기대빈도 < 1인 셀: {n_below_1}개")
cochran_ok = n_below_5 / len(expected_visits) <= 0.20 and n_below_1 == 0
print(f"    Cochran's rule: {'충족 ✓' if cochran_ok else '위반 ✗'}")

# (c) 적합도 검정
chi2_visits, p_visits = stats.chisquare(observed_visits, f_exp=expected_visits)

print(f"\n(c) 적합도 검정:")
print(f"    χ² = {chi2_visits:.4f}")
print(f"    자유도 (df) = {len(days) - 1}")
print(f"    p-value = {p_visits:.4f}")

# (d) 결론
print(f"\n(d) 결론 (α=0.05):")
if p_visits <= 0.05:
    print(f"    H₀ 기각 → 요일별 방문 비율이 균등하지 않습니다")
    print(f"    즉, 요일에 따라 방문 고객 수에 유의한 차이가 있습니다")
else:
    print(f"    H₀ 기각 실패 → 요일별 방문 비율이 균등하다는 증거가 부족합니다")


### 문제 3-2: 효과크기와 사후분석 (잔차 분석)

**(a)** 효과크기(Cohen's w)를 계산하고 해석하세요.

**(b)** 표준화 잔차를 계산하세요.

**각 요일**에 대해 잔차 값과 방향(기대보다 많음/적음)을 보고하세요.

> 표준화 잔차 = (O − E) / √E

**(c)** |잔차| > 2인 요일을 찾고, 이 요일들이 왜 기대와 다른지 실생활 관점에서 해석하세요.

**(d)** 관측 빈도와 기대 빈도를 비교하는 막대 그래프를 그리세요.

In [ ]:
print("\n[문제 3-2] 효과크기와 사후분석")
print("-" * 40)

# (a) Cohen's w
cohens_w_visits = np.sqrt(chi2_visits / n_total_visits)
w_label = '작은' if cohens_w_visits < 0.3 else '중간' if cohens_w_visits < 0.5 else '큰'

print(f"(a) Cohen's w = {cohens_w_visits:.4f}")
print(f"    해석 기준: w < 0.1 무시 / 0.1~0.3 작은 / 0.3~0.5 중간 / > 0.5 큰")
print(f"    해석: {w_label} 효과 크기입니다")

# (b) 표준화 잔차
print(f"\n(b) 사후분석 — 표준화 잔차:")
residuals = (observed_visits - expected_visits) / np.sqrt(expected_visits)
for i, d in enumerate(days):
    marker = " ⚠️" if abs(residuals[i]) > 2 else ""
    direction = "기대보다 많음 ↑" if residuals[i] > 0 else "기대보다 적음 ↓"
    print(f"    {d}: 잔차 = {residuals[i]:+.2f}  ({direction}){marker}")

# (c) 해석
print(f"\n(c) |잔차| > 2인 요일 해석:")
sig_days = [(days[i], residuals[i]) for i in range(len(days)) if abs(residuals[i]) > 2]
if sig_days:
    for day, res in sig_days:
        if res > 0:
            print(f"    {day}요일 (잔차={res:+.2f}): 기대보다 유의하게 많습니다")
            if day in ['토', '일', '금']:
                print(f"      → 주말/금요일은 외식 수요가 높아 방문이 증가하는 것으로 보입니다")
        else:
            print(f"    {day}요일 (잔차={res:+.2f}): 기대보다 유의하게 적습니다")
            if day in ['월', '화', '수']:
                print(f"      → 주 초반은 외식 수요가 낮아 방문이 감소하는 것으로 보입니다")
else:
    print(f"    |잔차| > 2인 요일이 없습니다")


In [ ]:
# (d) 관측 vs 기대 빈도 막대 그래프
fig, ax = plt.subplots(figsize=(10, 6))
x = np.arange(len(days))
width = 0.35

bars1 = ax.bar(x - width/2, observed_visits, width, color='#6366F1', alpha=0.8,
               edgecolor='white', label='관측 빈도')
bars2 = ax.bar(x + width/2, expected_visits, width, color='#F59E0B', alpha=0.8,
               edgecolor='white', label='기대 빈도 (균등)')

# 막대 위에 값 표시
for bar in bars1:
    ax.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 1,
            f'{int(bar.get_height())}', ha='center', va='bottom', fontsize=9)
for bar in bars2:
    ax.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 1,
            f'{bar.get_height():.0f}', ha='center', va='bottom', fontsize=9)

ax.set_xlabel('요일', fontsize=12)
ax.set_ylabel('방문 고객 수', fontsize=12)
ax.set_title(f'요일별 고객 방문 — 관측 vs 기대 (χ²={chi2_visits:.2f}, p={p_visits:.4f})',
             fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(days)
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()


### 문제 3-3: 소규모 카페 음료 선호도

위와 별개 상황입니다. 한 소규모 카페에서 5가지 음료의 선호도가 균등한지 조사했습니다.

총 **22명**의 고객이 응답했습니다.

**분석 목표**: 5가지 음료의 선호도가 균등(각 1/5)한지 검정합니다.

**주어진 데이터:**

In [ ]:
print("\n[문제 3-3] 소규모 카페 음료 선호도")
print("-" * 40)

menu = ['아메리카노', '라떼', '카푸치노', '스무디', '에이드']
observed_small = np.array([8, 5, 4, 3, 2])
n_total_small = observed_small.sum()

print(f"음료별 선호 고객 수 (총 {n_total_small}명):")
df_small = pd.DataFrame({'음료': menu, '관측 빈도': observed_small})
display(df_small)


**(a)** 가설을 설정하세요.

**(b)** 균등 분포를 가정할 때 기대빈도를 계산하고, Cochran's rule을 점검하세요.

조건이 충족되나요?

**(c)** Cochran's rule이 위반되면, 사용할 수 있는 대안 방법은 무엇인가요?

범주 병합과 Monte Carlo 시뮬레이션 중 어떤 것이 이 상황에 더 적절한지 설명하세요.

**(d)** 선택한 대안 방법을 적용하여 검정을 수행하세요.

**(e)** 효과크기(Cohen's w)를 계산하고 해석하세요.

In [ ]:
# (a) 가설 설정
print("(a) 가설 설정:")
print(f"    H₀: 5가지 음료의 선호도가 균등합니다 (p₁ = p₂ = p₃ = p₄ = p₅ = 1/5)")
print(f"    H₁: 적어도 하나의 음료가 다른 선호도를 보입니다")

# (b) 기대빈도 + Cochran's rule
expected_small = np.full(len(menu), n_total_small / len(menu))

print(f"\n(b) 기대빈도: {expected_small[0]:.1f} (모든 음료 동일)")
n_below_5_s = np.sum(expected_small < 5)
pct_below_5_s = n_below_5_s / len(expected_small) * 100
n_below_1_s = np.sum(expected_small < 1)
print(f"    기대빈도 < 5인 셀: {n_below_5_s}개 ({pct_below_5_s:.0f}%)")
print(f"    기대빈도 < 1인 셀: {n_below_1_s}개")
cochran_ok_s = n_below_5_s / len(expected_small) <= 0.20 and n_below_1_s == 0
print(f"    Cochran's rule: {'충족 ✓' if cochran_ok_s else '위반 ✗ → 모든 셀의 기대빈도가 5 미만입니다'}")

# (c) 대안 방법 선택
print(f"\n(c) 대안 방법:")
print(f"    (1) 범주 병합: '카푸치노', '스무디', '에이드'를 '기타'로 병합 가능")
print(f"       → 하지만 서로 성격이 다른 음료를 합치면 해석이 왜곡될 수 있습니다")
print(f"    (2) Monte Carlo 시뮬레이션: 무작위 재표본을 반복하여 p-value를 추정")
print(f"       → 범주의 의미를 유지하면서 검정할 수 있어 이 상황에 더 적절합니다")
print(f"    → 선택: Monte Carlo 시뮬레이션 (+ 참고용으로 범주 병합도 수행)")

# (d) 검정 수행 — Monte Carlo 시뮬레이션
np.random.seed(503)
chi2_obs = np.sum((observed_small - expected_small)**2 / expected_small)

n_sim = 10000
chi2_sim = np.zeros(n_sim)
for i in range(n_sim):
    sim_data = np.random.multinomial(n_total_small, [1/len(menu)] * len(menu))
    chi2_sim[i] = np.sum((sim_data - expected_small)**2 / expected_small)

p_monte_carlo = np.mean(chi2_sim >= chi2_obs)

print(f"\n(d-1) Monte Carlo 시뮬레이션 결과 (N={n_sim:,}회):")
print(f"    관측 χ² = {chi2_obs:.4f}")
print(f"    Monte Carlo p-value = {p_monte_carlo:.4f}")
verdict_mc = "H₀ 기각" if p_monte_carlo <= 0.05 else "H₀ 기각 실패"
print(f"    판정 (α=0.05): {verdict_mc}")

# 참고: 범주 병합
merged_obs = np.array([8, 5, 4 + 3 + 2])  # 아메리카노, 라떼, 기타
merged_exp = np.array([n_total_small / 3] * 3)  # 3개 범주 균등
chi2_merged, p_merged = stats.chisquare(merged_obs, f_exp=merged_exp)

print(f"\n(d-2) [참고] 범주 병합 결과 (카푸치노+스무디+에이드 → 기타):")
print(f"    병합 후: 아메리카노={merged_obs[0]}, 라떼={merged_obs[1]}, 기타={merged_obs[2]}")
print(f"    χ² = {chi2_merged:.4f}, p = {p_merged:.4f}")

p_alternative = p_monte_carlo

# (e) 효과크기
cohens_w_small = np.sqrt(chi2_obs / n_total_small)
w_label_s = '작은' if cohens_w_small < 0.3 else '중간' if cohens_w_small < 0.5 else '큰'

print(f"\n(e) Cohen's w = {cohens_w_small:.4f}")
print(f"    해석 기준: w < 0.1 무시 / 0.1~0.3 작은 / 0.3~0.5 중간 / > 0.5 큰")
print(f"    해석: {w_label_s} 효과 크기입니다")



---

## 문제 4: 연령대별 운동 선호도 조사

헬스장에서 연령대(20대/30대/40대)에 따라 선호하는 운동 종류(헬스/요가/수영)가 다른지 조사합니다.

총 **180명**을 대상으로 설문을 실시했습니다.

**분석 목표**: 연령대에 따라 선호하는 운동 종류에 유의한 차이가 있는지 검정합니다.

**주어진 데이터:**

In [ ]:
print("\n[문제 4] 연령대별 운동 선호도 조사")
print("=" * 50)

np.random.seed(504)
n_survey = 180

ages = np.random.choice(['20대', '30대', '40대'], size=n_survey, p=[0.35, 0.35, 0.30])

exercise_prefs = []
for age in ages:
    if age == '20대':
        exercise_prefs.append(np.random.choice(['헬스', '요가', '수영'], p=[0.50, 0.20, 0.30]))
    elif age == '30대':
        exercise_prefs.append(np.random.choice(['헬스', '요가', '수영'], p=[0.30, 0.40, 0.30]))
    else:
        exercise_prefs.append(np.random.choice(['헬스', '요가', '수영'], p=[0.20, 0.35, 0.45]))
exercise_prefs = np.array(exercise_prefs)

df_survey = pd.DataFrame({'연령대': ages, '운동': exercise_prefs})

ct_exercise = pd.crosstab(df_survey['연령대'], df_survey['운동'],
                            margins=True, margins_name='합계')
print("\n교차표:")
display(ct_exercise)


### 문제 4-1: 가설 설정 및 독립성 검정

**(a)** 분석 목표에 맞는 **귀무가설(H₀)**과 **대립가설(H₁)**을 설정하세요.

> 카이제곱 독립성 검정은 양측/단측 개념이 적용되지 않습니다.

**(b)** 기대빈도를 계산하고 표로 출력하세요.

Cochran's rule(기대빈도 < 5인 셀이 20% 이하, 기대빈도 < 1인 셀이 0개)을 점검하세요.

**(c)** 조건 충족 여부에 따라 적절한 검정을 수행하세요 (α = 0.05).

**(d)** 결론을 내리세요.

In [ ]:
print("\n[문제 4-1] 가설 설정 및 독립성 검정")
print("-" * 40)

ct_raw_ex = pd.crosstab(df_survey['연령대'], df_survey['운동'])

# (a) 가설 설정
print("(a) 가설 설정:")
print(f"    H₀: 연령대와 운동 선호는 독립입니다 (연관이 없습니다)")
print(f"    H₁: 연령대와 운동 선호는 독립이 아닙니다 (연관이 있습니다)")
print(f"    * 카이제곱 독립성 검정은 양측/단측 개념이 적용되지 않습니다")

# (b) 기대빈도 + Cochran's rule
chi2_ex, p_ex, dof_ex, expected_ex = stats.chi2_contingency(ct_raw_ex)

print(f"\n(b) 기대빈도:")
expected_df_ex = pd.DataFrame(expected_ex.round(1),
                               index=ct_raw_ex.index, columns=ct_raw_ex.columns)
display(expected_df_ex)

total_cells = expected_ex.size
n_below_5_ex = np.sum(expected_ex < 5)
n_below_1_ex = np.sum(expected_ex < 1)
pct_below_5_ex = n_below_5_ex / total_cells * 100
print(f"    전체 셀 수: {total_cells}")
print(f"    기대빈도 < 5인 셀: {n_below_5_ex}개 ({pct_below_5_ex:.0f}%)")
print(f"    기대빈도 < 1인 셀: {n_below_1_ex}개")
cochran_ok_ex = pct_below_5_ex <= 20 and n_below_1_ex == 0
print(f"    Cochran's rule: {'충족 ✓ → 카이제곱 검정 사용 가능' if cochran_ok_ex else '위반 ✗'}")

# (c) 검정 수행
print(f"\n(c) 독립성 검정:")
print(f"    χ² = {chi2_ex:.4f}")
print(f"    자유도 (df) = {dof_ex}")
print(f"    p-value = {p_ex:.4f}")

# (d) 결론
print(f"\n(d) 결론 (α=0.05):")
if p_ex <= 0.05:
    print(f"    H₀ 기각 → 연령대와 운동 선호는 독립이 아닙니다")
    print(f"    즉, 연령대에 따라 운동 선호도에 유의한 차이가 있습니다")
else:
    print(f"    H₀ 기각 실패 → 연령대와 운동 선호가 독립이라는 증거가 부족합니다")


### 문제 4-2: 효과크기와 사후분석 (조정된 잔차)

**(a)** Cramér's V를 계산하고 해석하세요.

df* = min(r−1, c−1)에 따른 해석 기준을 적용하세요.

**(b)** 조정된 잔차(표준화 잔차)를 구하고 표로 출력하세요.

**(c)** |잔차| > 2인 셀을 찾고, 각 셀이 의미하는 바를 구체적으로 해석하세요.
> 예: "20대 × 헬스: 잔차 = +2.5 → 20대는 기대보다 헬스를 더 많이 선호합니다"

**(d)** 관측 빈도와 기대 빈도를 비교하는 히트맵을 나란히 그리세요.

In [ ]:
print("\n[문제 4-2] 효과크기와 사후분석")
print("-" * 40)

# (a) Cramér's V
r_ex, c_ex = ct_raw_ex.shape
df_star_ex = min(r_ex - 1, c_ex - 1)
cramers_v_ex = np.sqrt(chi2_ex / (n_survey * df_star_ex))

# df*에 따른 해석 기준
small_v = 0.1 / np.sqrt(df_star_ex)
medium_v = 0.3 / np.sqrt(df_star_ex)
large_v = 0.5 / np.sqrt(df_star_ex)

if cramers_v_ex < medium_v:
    v_label = '작은'
elif cramers_v_ex < large_v:
    v_label = '중간'
else:
    v_label = '큰'

print(f"(a) Cramér's V = {cramers_v_ex:.4f}")
print(f"    df* = {df_star_ex}")
print(f"    해석 기준 (df*={df_star_ex}): 작은 {small_v:.3f} / 중간 {medium_v:.3f} / 큰 {large_v:.3f}")
print(f"    해석: {v_label} 효과 크기입니다")

# (b) 조정된 잔차
table_ex = Table(ct_raw_ex)
std_res_ex = pd.DataFrame(table_ex.standardized_resids,
                           index=ct_raw_ex.index, columns=ct_raw_ex.columns)

print(f"\n(b) 조정된 잔차:")
display(std_res_ex.round(2))

# (c) |잔차| > 2인 셀 해석
print(f"\n(c) |잔차| > 2인 셀:")
sig_cells_ex = std_res_ex.stack()
sig_cells_ex = sig_cells_ex[sig_cells_ex.abs() > 2]
if len(sig_cells_ex) > 0:
    for (row, col), val in sig_cells_ex.items():
        direction = "기대보다 많음 ↑" if val > 0 else "기대보다 적음 ↓"
        print(f"    {row} × {col}: 잔차={val:.2f} → {direction}")
        if val > 0:
            print(f"      → {row}는 {col}을(를) 기대보다 유의하게 더 많이 선호합니다")
        else:
            print(f"      → {row}는 {col}을(를) 기대보다 유의하게 적게 선호합니다")
else:
    print(f"    |잔차| > 2인 셀이 없습니다")


In [ ]:
# (d) 히트맵 시각화 (관측 빈도 vs 기대 빈도)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ct_obs = ct_raw_ex.values
im1 = axes[0].imshow(ct_obs, cmap='Blues', aspect='auto')
axes[0].set_title('관측 빈도', fontsize=13, fontweight='bold')
axes[0].set_xticks(range(c_ex))
axes[0].set_xticklabels(ct_raw_ex.columns)
axes[0].set_yticks(range(r_ex))
axes[0].set_yticklabels(ct_raw_ex.index)
for i in range(r_ex):
    for j in range(c_ex):
        axes[0].text(j, i, f'{ct_obs[i, j]}', ha='center', va='center',
                     fontsize=14, fontweight='bold',
                     color='white' if ct_obs[i, j] > ct_obs.max() * 0.6 else 'black')
fig.colorbar(im1, ax=axes[0], shrink=0.8)

im2 = axes[1].imshow(expected_ex, cmap='Oranges', aspect='auto')
axes[1].set_title('기대 빈도 (독립 가정)', fontsize=13, fontweight='bold')
axes[1].set_xticks(range(c_ex))
axes[1].set_xticklabels(ct_raw_ex.columns)
axes[1].set_yticks(range(r_ex))
axes[1].set_yticklabels(ct_raw_ex.index)
for i in range(r_ex):
    for j in range(c_ex):
        axes[1].text(j, i, f'{expected_ex[i, j]:.1f}', ha='center', va='center',
                     fontsize=14, fontweight='bold',
                     color='white' if expected_ex[i, j] > expected_ex.max() * 0.6 else 'black')
fig.colorbar(im2, ax=axes[1], shrink=0.8)

plt.tight_layout()
plt.show()


### 문제 4-3: 면접 방식별 합격률

위 데이터와 별개로, 한 중소기업 인사팀에서 **면접 방식**(대면/화상)에 따라

최종 합격률이 다른지 파악하려 합니다.

소규모 채용 회차로 총 **20명**의 지원자 데이터입니다.

**분석 목표**: 면접 방식(대면/화상)에 따라 최종 합격률에 유의한 차이가 있는지 검정합니다.

**주어진 데이터:**

In [ ]:
print("\n[문제 4-3] 면접 방식별 합격률")
print("-" * 40)

# 2×2 교차표
#              합격    불합격
# 대면 면접     7       3      = 10
# 화상 면접     2       8      = 10
#               9      11      = 20
data_fisher_q4 = np.array([[7, 3],
                             [2, 8]])

ct_fisher_q4 = pd.DataFrame(data_fisher_q4,
                              index=['대면 면접', '화상 면접'],
                              columns=['합격', '불합격'])
ct_display_q4 = ct_fisher_q4.copy()
ct_display_q4['합계'] = ct_display_q4.sum(axis=1)
ct_display_q4.loc['합계'] = ct_display_q4.sum()
print("교차표:")
display(ct_display_q4)


**(a)** **귀무가설(H₀)**과 **대립가설(H₁)**을 설정하세요.

**(b)** **양측검정** 또는 **단측검정** 중 어떤 것이 적절한지 결정하고, 그 근거를 설명하세요.

> Fisher 정확검정에서는 양측/단측 선택이 결과에 직접 영향을 줍니다.

**(c)** 기대빈도를 계산하고 Cochran's rule을 점검하세요.

카이제곱 검정을 적용할 수 있는지 판단하세요.

**(d)** 적절한 검정을 선택하여 수행하세요 (α = 0.05).

왜 이 검정을 선택했는지 이유를 설명하세요.

**(e)** 오즈비(OR)를 구하고 해석하세요.

> 오즈(Odds) = 해당 사건 / 반대 사건 \
> 오즈비(OR) = 한 집단의 오즈 / 다른 집단의 오즈 \
> 예: 대면 면접의 합격 오즈 = 합격 / 불합격

**(f)** 효과크기 φ(Phi) 계수를 계산하고 해석하세요.

In [ ]:
# (a) 가설 설정
print("(a) 가설 설정:")
print(f"    H₀: 면접 방식과 합격 여부는 독립입니다 (면접 방식에 따른 합격률 차이가 없습니다)")
print(f"    H₁: 면접 방식과 합격 여부는 독립이 아닙니다 (면접 방식에 따라 합격률이 다릅니다)")

# (b) 양측/단측 결정
print(f"\n(b) 검정 방향:")
print(f"    선택: 양측검정")
print(f"    근거: 대면과 화상 중 어느 방식이 더 유리한지 사전에 알 수 없으므로")
print(f"    방향성을 특정할 근거가 없습니다. 따라서 양측검정이 적절합니다.")

# (c) 기대빈도 + Cochran's rule
# 효과크기(Phi) 계산에는 Yates 보정 없는 Pearson χ²를 사용해야 합니다
chi2_q4, p_chi2_q4, dof_q4, expected_q4 = stats.chi2_contingency(data_fisher_q4, correction=False)

print(f"\n(c) 기대빈도:")
expected_df_q4 = pd.DataFrame(expected_q4,
                                index=['대면 면접', '화상 면접'],
                                columns=['합격', '불합격'])
display(expected_df_q4)

total_cells_q4 = expected_q4.size
n_below_5_q4 = np.sum(expected_q4 < 5)
pct_below_5_q4 = n_below_5_q4 / total_cells_q4 * 100
n_below_1_q4 = np.sum(expected_q4 < 1)
print(f"    기대빈도 < 5인 셀: {n_below_5_q4}개 ({pct_below_5_q4:.0f}%)")
print(f"    기대빈도 < 1인 셀: {n_below_1_q4}개")
cochran_ok_q4 = pct_below_5_q4 <= 20 and n_below_1_q4 == 0
print(f"    Cochran's rule: {'충족 ✓' if cochran_ok_q4 else '위반 ✗'}")
if not cochran_ok_q4:
    print(f"    → 카이제곱 근사가 부정확합니다. Fisher 정확검정을 사용해야 합니다")
else:
    print(f"    → 그러나 N={data_fisher_q4.sum()}으로 소표본이므로 Fisher 정확검정이 더 적절합니다")

# (d) Fisher 정확검정
odds_ratio_q4, fisher_p_q4 = stats.fisher_exact(data_fisher_q4)

print(f"\n(d) Fisher 정확검정:")
print(f"    p-value = {fisher_p_q4:.4f}")
verdict_fisher = "H₀ 기각 → 면접 방식에 따라 합격률이 다릅니다" if fisher_p_q4 <= 0.05 else "H₀ 기각 실패 → 유의한 차이가 없습니다"
print(f"    판정 (α=0.05): {verdict_fisher}")
print(f"    검정 선택 이유: 기대빈도 < 5인 셀이 {pct_below_5_q4:.0f}%로, Cochran's rule이 위반되어")
print(f"    카이제곱 근사가 부정확하므로 Fisher 정확검정을 사용했습니다")

# (e) 오즈비 해석
odds_face = 7 / 3
odds_video = 2 / 8
print(f"\n(e) 오즈비:")
print(f"    대면 면접 합격 오즈 = 7/3 = {odds_face:.4f}")
print(f"    화상 면접 합격 오즈 = 2/8 = {odds_video:.4f}")
print(f"    OR = {odds_face:.4f} / {odds_video:.4f} = {odds_ratio_q4:.4f}")
if odds_ratio_q4 > 1:
    print(f"    해석: 대면 면접의 합격 오즈가 화상 면접보다 {odds_ratio_q4:.1f}배 높습니다")
else:
    print(f"    해석: 화상 면접의 합격 오즈가 대면 면접보다 {1/odds_ratio_q4:.1f}배 높습니다")

# (f) Phi 계수
N_q4 = data_fisher_q4.sum()
phi_q4 = np.sqrt(chi2_q4 / N_q4)
phi_label = '작은' if phi_q4 < 0.3 else '중간' if phi_q4 < 0.5 else '큰'

print(f"\n(f) Phi 계수(φ) = {phi_q4:.4f}")
print(f"    해석 기준: φ < 0.1 무시 / 0.1~0.3 작은 / 0.3~0.5 중간 / > 0.5 큰")
print(f"    해석: {phi_label} 효과 크기입니다")



---

### 핵심 요약

| 개념                   | 한 줄 요약                                                                     |
|------------------------|--------------------------------------------------------------------------------|
| **가설검정 프로세스**  | 분석 목표 → 가설 설정 → 검정 방향 → 가정 검정 → 검정 수행 → 효과크기 → 결론  |
| **정규성 검정**        | Shapiro-Wilk(수치) + Q-Q Plot(시각)을 함께 사용하여 분포를 판단합니다          |
| **비모수 검정**        | 정규성 위반 시 순위 기반 검정(Wilcoxon, Mann-Whitney U)을 사용합니다           |
| **효과크기 (비모수)**  | rank-biserial r로 효과의 실질적 크기를 보고합니다 (0.1/0.3/0.5)               |
| **카이제곱 적합도**    | 관측 빈도가 이론적 비율과 일치하는지 검정합니다 (범주형 1변수)                 |
| **카이제곱 독립성**    | 두 범주형 변수가 서로 독립인지 검정합니다 (교차표)                             |
| **Cochran's rule**     | 기대빈도 < 5인 셀이 20% 초과 시 정확검정 또는 범주 병합이 필요합니다          |
| **사후분석 (잔차)**    | 표준화 잔차 |값| > 2인 범주/셀이 기대빈도와 유의하게 다릅니다               |
| **Fisher 정확검정**    | 소표본·기대빈도 부족 시 근사 대신 정확한 확률을 계산합니다 (오즈비 함께 산출)  |
| **오즈비 (OR)**        | OR > 1이면 한 집단의 사건 발생 오즈가 더 크다는 뜻입니다                       |

In [ ]:
print("\n" + "=" * 60)
print("정답 요약")
print("=" * 60)

print(f"\n[문제 1] 편의점 매출")
print(f"  가설: H₀: μ = 250 / H₁: μ ≠ 250 (양측검정)")
print(f"  Shapiro-Wilk: W={stat_sw:.4f}, p={p_sw:.4f}")
print(f"  Wilcoxon: W={test_stat:.1f}, p={p_value:.4f}")
print(f"  rank-biserial r = {effect_size:.4f}")
print(f"  t-검정: t={t_stat_1:.4f}, p={p_t:.4f}")

print(f"\n[문제 2] 생산라인")
print(f"  가설: H₀: μ_A = μ_B / H₁: μ_A ≠ μ_B (양측검정)")
print(f"  정규성: A(p={p_a:.4f}), B(p={p_b:.4f})")
print(f"  등분산: p={lev_p:.4f}")
print(f"  검정: p={p_value_2:.4f}, 효과크기={effect_size_2:.4f}")
print(f"  Welch's t: p={p_t_comp:.4f} / Mann-Whitney: p={p_u_comp:.4f}")

print(f"\n[문제 3] 요일별 방문")
print(f"  가설: H₀: 모든 pᵢ = 1/7 / H₁: 적어도 하나의 pᵢ ≠ 1/7")
print(f"  적합도: χ²={chi2_visits:.4f}, p={p_visits:.4f}")
print(f"  Cohen's w = {cohens_w_visits:.4f}")
print(f"  소규모 Monte Carlo: p={p_monte_carlo:.4f}")

print(f"\n[문제 4] 운동 선호도")
print(f"  가설: H₀: 연령대와 운동 독립 / H₁: 독립 아님")
print(f"  독립성: χ²={chi2_ex:.4f}, p={p_ex:.4f}")
print(f"  Cramér's V = {cramers_v_ex:.4f}")
print(f"  Fisher (면접): H₀: 독립 / H₁: 독립 아님 (양측검정)")
print(f"    p={fisher_p_q4:.4f}, OR={odds_ratio_q4:.4f}")
print(f"  Phi = {phi_q4:.4f}")
